In [1]:
# Training script for LazyConfig models
try:
    # ignore ShapelyDeprecationWarning from fvcore
    import warnings

    from shapely.errors import ShapelyDeprecationWarning

    warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning)

except:
    pass
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=UserWarning)

# Some basic setup:
# Setup detectron2 logger
from detectron2.utils.logger import setup_logger

setup_logger()

import gc
import os
import time

import detectron2.utils.comm as comm

# import some common libraries
import numpy as np
import torch

# import some common detectron2 utilities
from detectron2.config import LazyConfig, get_cfg
from detectron2.engine import (
    launch,
)

from deepdisc.data_format.augment_image import train_augs
from deepdisc.data_format.image_readers import DC2ImageReader
from deepdisc.data_format.register_data import register_data_set
from deepdisc.model.loaders import (
    RedshiftFlatDictMapper,
    return_test_loader,
    return_train_loader,
)
from deepdisc.model.models import RedshiftPointCasROIHeads, RedshiftPointROIHeads, RedshiftPDFROIHeads, return_lazy_model
from deepdisc.training.trainers import (
    return_evallosshook,
    return_lazy_trainer,
    return_optimizer,
    return_savehook,
    return_schedulerhook,
)
from deepdisc.utils.parse_arguments import make_training_arg_parser


/home/g4merz/.conda/envs/deepdiscnv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
cfgfile = "/home/shared/hsc/detectron2/configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_1x.py"
dirpath = './tests/deepdisc/test_data/'
output_dir = './'
output_name = 'test'

trainfile = dirpath + "flattened_data_test.npy"
testfile = dirpath + "flattened_data_test.npy"


In [8]:
classes = ['object']
numclasses = 1

testdata = np.load('./tests/deepdisc/test_data/flattened_data_test.npy')


mapper = RedshiftFlatDictMapper().map_data
dataset_dicts={}
dds = []
for row in testdata:
    dds.append(mapper(row))
dataset_dicts['test'] = dds


In [9]:
astrotrain_metadata = register_data_set("astro_train", trainfile, np.load, thing_classes=classes)
astroval_metadata = register_data_set("astro_val", testfile, np.load, thing_classes=classes)

In [14]:
cfg = LazyConfig.load(cfgfile)

# metadata = MetadataCatalog.get(cfg.dataloader.test.dataset.names) # to get labels from ids

bs = 1
cfg.model.proposal_generator.anchor_generator.sizes = [[8], [16], [32], [64], [128]]
cfg.model.proposal_generator.batch_size_per_image = 512

cfg.dataloader.train.total_batch_size = bs
cfg.model.roi_heads.num_classes = numclasses
cfg.model.roi_heads.batch_size_per_image = 128
#cfg.model.backbone.bottom_up.in_chans = 6
cfg.model.backbone.bottom_up.stem.in_channels = 6
#cfg.model.pixel_mean = [0.00376413, 0.01292477, 0.04308919, 0.08965224, 0.13665777, 0.16544563]
#cfg.model.pixel_std =[0.07028706, 0.06942783, 0.2432113, 0.55617297, 0.8046262, 0.9044935]
cfg.model.pixel_mean = [0.00604957, 0.0097893, 0.02428164, 0.04782381, 0.06689043, 0.07634047]
cfg.model.pixel_std = [0.07637636, 0.07277052, 0.19012378, 0.42937884, 0.5813913, 0.65684605]

cfg.model.roi_heads.num_components = 1
#cfg.model.roi_heads._target_ = RedshiftPointCasROIHeads
cfg.model.roi_heads._target_ = RedshiftPDFROIHeads

cfg_loader = get_cfg()
cfg_loader.SOLVER.IMS_PER_BATCH = bs
cfg_loader.DATASETS.TRAIN = "astro_train"  # Register Metadata
cfg_loader.DATASETS.TEST = "astro_val"
# cfg_loader.DATALOADER.NUM_WORKERS = 0
#cfg_loader.DATALOADER.PREFETCH_FACTOR = 2
cfg_loader.SOLVER.BASE_LR = 0.001
cfg_loader.OUTPUT_DIR = output_dir
os.makedirs(cfg_loader.OUTPUT_DIR, exist_ok=True)

cfg_loader.SOLVER.CLIP_GRADIENTS.ENABLED = True
# Type of gradient clipping, currently 2 values are supported:
# - "value": the absolute values of elements of each gradients are clipped
# - "norm": the norm of the gradient for each parameter is clipped thus
#   affecting all elements in the parameter
cfg_loader.SOLVER.CLIP_GRADIENTS.CLIP_TYPE = "norm"
# Maximum absolute value used for clipping gradients
# Floating point number p for L-p norm to be used with the "norm"
# gradient clipping type; for L-inf, please specify .inf
cfg_loader.SOLVER.CLIP_GRADIENTS.NORM_TYPE = 5.0

# iterations for 15,25,35,50 epochs
epoch = int(len(dataset_dicts['test']) / cfg.dataloader.train.total_batch_size)
e1 = 200
e2 = epoch * 10
e3 = epoch * 20
efinal = epoch * 35

val_per = 5


In [16]:
model = return_lazy_model(cfg)

cfg.optimizer.params.model = model
cfg.optimizer.lr = 0.001

cfg_loader.SOLVER.STEPS = []  # do not decay learning rate for retraining
cfg_loader.SOLVER.LR_SCHEDULER_NAME = "WarmupMultiStepLR"
cfg_loader.SOLVER.WARMUP_ITERS = 0
cfg_loader.SOLVER.MAX_ITER = e1  # for DefaultTrainer

# optimizer = instantiate(cfg.optimizer)

optimizer = return_optimizer(cfg)

mapper = RedshiftFlatDictMapper().map_data
loader = return_train_loader(cfg_loader, mapper)
test_loader = return_test_loader(cfg_loader, mapper)

saveHook = return_savehook(output_name)
lossHook = return_evallosshook(val_per, model, test_loader)
schedulerHook = return_schedulerhook(optimizer)
hookList = [lossHook, schedulerHook, saveHook]

trainer = return_lazy_trainer(model, loader, optimizer, cfg, cfg_loader, hookList)

trainer.set_period(5)
trainer.train(0, e1)

np.save(output_dir + output_name + "_losses", trainer.lossList)
np.save(output_dir + output_name + "_val_losses", trainer.vallossList)

[11/09 10:50:23 d2.data.build]: Using training sampler TrainingSampler
[11/09 10:50:23 d2.data.common]: Serializing the dataset using: <class 'detectron2.data.common._TorchSerializedList'>
[11/09 10:50:23 d2.data.common]: Serializing 10 elements to byte tensors and concatenating them all ...
[11/09 10:50:23 d2.data.common]: Serialized dataset takes 7.50 MiB
[11/09 10:50:23 d2.data.common]: Serializing the dataset using: <class 'detectron2.data.common._TorchSerializedList'>
[11/09 10:50:23 d2.data.common]: Serializing 10 elements to byte tensors and concatenating them all ...
[11/09 10:50:23 d2.data.common]: Serialized dataset takes 7.50 MiB
[11/09 10:50:23 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from detectron2://ImageNetPretrained/MSRA/R-50.pkl ...
[11/09 10:50:23 d2.checkpoint.c2_model_loading]: Renaming Caffe2 weights ......
WARNING [11/09 10:50:23 d2.checkpoint.c2_model_loading]: Shape of stem.conv1.weight in checkpoint is torch.Size([64, 3, 7, 7]), whi

/home/shared/hsc/detectron2/detectron2/data/build.py:264: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  has_instances = "annotations" in dataset_dicts[0]
Some model parameters or buffers are not found in the checkpoint:
backbone.bottom_up.stem.conv1.weight
backbone.fpn_lateral2.{bias, weight}
backbone.fpn_lateral3.{bias, weight}
backbone.fpn_lateral4.{bias, weight}
backbone.fpn_lateral5.{bias, weight}
backbone.fpn_output2.{bias, weight}
backbone.fpn_output3.{bias, weight}
backbone.fpn_output4.{bias, weight}
backbone.fpn_output5.{bias, weight}
proposal_generator.rpn_head.anchor_deltas.{bias, weight}
proposal_generator.rpn_head.conv.{bias, weight}
proposal_generator.rpn_head.objectness_logits.{bias, weight}
roi_heads.box_head.fc1.{bias, weight}
roi_heads.box_head.fc2.{bias, weight}
roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, weight}
roi_heads.mask_head.deconv.{b

[11/09 10:50:23 d2.engine.train_loop]: Starting training from iteration 0
Iteration:  5  time:  2.8870999813079834e-07 dict_keys(['loss_cls', 'loss_box_reg', 'redshift_loss', 'loss_rpn_cls', 'loss_rpn_loc']) [0.17706850171089172, 0.09543279558420181, 0.07290288359491061, 0.5679972171783447, 0.001256515271961689] val loss:  0 lr:  [0.001]
Iteration:  10  time:  3.03611159324646e-07 dict_keys(['loss_cls', 'loss_box_reg', 'redshift_loss', 'loss_rpn_cls', 'loss_rpn_loc']) [0.2060682475566864, 0.08948379755020142, 0.03264159844622285, 0.2916221618652344, 0.0013081433717161417] val loss:  0.7717061652065942 lr:  [0.001]
Iteration:  15  time:  2.5425106287002563e-07 dict_keys(['loss_cls', 'loss_box_reg', 'redshift_loss', 'loss_rpn_cls', 'loss_rpn_loc']) [0.07782269269227982, 0.07685889303684235, 0.13704210288681884, 0.06937579810619354, 0.0017682438483461738] val loss:  0.46683182052372096 lr:  [0.001]
Iteration:  20  time:  2.2351741790771484e-07 dict_keys(['loss_cls', 'loss_box_reg', 'redsh

### Inference

In [56]:
import importlib
import deepdisc
importlib.reload(deepdisc.model.models)

<module 'deepdisc.model.models' from '/home/g4merz/deepdisc/src/deepdisc/model/models.py'>

In [18]:
from detectron2.config import LazyConfig
from deepdisc.model.models import  RedshiftPointCasROIHeads, RedshiftPointROIHeads, RedshiftPDFROIHeads, return_lazy_model
from deepdisc.inference.predictors import return_predictor_transformer

roi_thresh = 0.4
output_dir = './'

cfgfile = "/home/shared/hsc/detectron2/configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_1x.py"

cfg = LazyConfig.load(cfgfile)

#metadata = MetadataCatalog.get(cfg.dataloader.test.dataset.names)  # to get labels from ids
#classes = metadata.thing_classes

cfg.model.proposal_generator.anchor_generator.sizes = [[8], [16], [32], [64], [128]]
cfg.model.proposal_generator.batch_size_per_image = 512

#cfg.dataloader.train.total_batch_size = 4
cfg.model.roi_heads.num_classes = 1
cfg.model.roi_heads.batch_size_per_image = 128
cfg.model.backbone.bottom_up.stem.in_channels = 6
cfg.model.pixel_mean = [0.00604957, 0.0097893, 0.02428164, 0.04782381, 0.06689043, 0.07634047]
cfg.model.pixel_std = [0.07637636, 0.07277052, 0.19012378, 0.42937884, 0.5813913, 0.65684605]
cfg.model.roi_heads.num_components = 1
#cfg.model.roi_heads._target_ = NewRedshiftPDFCasROIHeads
cfg.model.roi_heads._target_ = RedshiftPDFROIHeads

cfg.model.roi_heads.box_predictor.test_score_thresh = roi_thresh
#for bp in cfg.model.roi_heads.box_predictors:
#    bp.test_score_thresh = roi_thresh
#cfg.model.proposal_generator.pre_nms_topk = [6000, 6000]
#cfg.model.proposal_generator.post_nms_topk = [6000, 6000]
cfg.model.proposal_generator.nms_thresh = 0.3

#for box_predictor in cfg.model.roi_heads.box_predictors:
#    #box_predictor.test_topk_per_image = 1000
#    box_predictor.test_score_thresh = roi_thresh

cfg_loader = get_cfg()
cfg_loader.SOLVER.IMS_PER_BATCH = 4
cfg_loader.DATASETS.TRAIN = "astro_test"  # Register Metadata
cfg_loader.DATASETS.TEST = "astro_test"
cfg_loader.DATALOADER.NUM_WORKERS = 1
cfg_loader.SOLVER.BASE_LR = 0.001
cfg_loader.OUTPUT_DIR = output_dir

cfg_loader.TEST.DETECTIONS_PER_IMAGE = 128

cfg_loader.OUTPUT_DIR = output_dir

cfg.train.init_checkpoint = os.path.join('./test.pth')

predictor = return_predictor_transformer(cfg, cfg_loader)


[11/09 10:53:49 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from ./test.pth ...


In [21]:
from deepdisc.inference.match_objects import get_matched_object_classes_new, get_matched_z_pdfs_new


In [23]:
print("Matching objects")
true_classes, pred_classes = get_matched_object_classes_new(dataset_dicts["test"],  predictor)
true_zs, pred_pdfs = get_matched_z_pdfs_new(dataset_dicts["test"], predictor)



Matching objects
